## 训练

In [1]:
import torch
from codes.model import ResUnet_LSTM_L, Loss_With_Weight
from torch.utils.data import DataLoader
from codes.dataloader import init_dataset, STEAD_Dataset
from codes.visualize_and_evaluate import evaluate
from codes import configs

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

train_set,val_set,noise_set = init_dataset()
train_set = STEAD_Dataset(train_set)
val_set = STEAD_Dataset(val_set)

train_dataloader = DataLoader(train_set, shuffle=True, batch_size=configs.BATCH_SIZE, num_workers=32, drop_last=True)
eval_dataloader = DataLoader(val_set, shuffle=True,batch_size=1, num_workers=16, drop_last=True)

mseLoss = torch.nn.MSELoss()
#QuackNet = ResUnet().to(device)# 不带lstm的版本
QuackNet = ResUnet_LSTM_L(True).to(device)# 带lstm的版本

optimizer = torch.optim.Adam(QuackNet.parameters(),
                lr=configs.LEARNING_RATE,
                betas=(0.9, 0.999),
                eps=1e-08,
                weight_decay=0,
                amsgrad=False)

total_step = 0
loss_sum = 0
loss_list = []
for epoch in range(configs.EPOCH_NUM):
    # training
    for index,data in enumerate(train_dataloader,0):
        stream,label_p,label_s,p_start,s_start,coda_end = data

        # 清空累加梯度
        QuackNet.zero_grad()
        output = QuackNet(stream.to(device))
        loss = 0
        loss += Loss_With_Weight(output[:,0,:], label_p.to(device))
        loss += Loss_With_Weight(output[:,1,:], label_s.to(device))
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        total_step += 1
        if total_step%configs.LOSS_RECORD_ITER == 0:
            loss_list.append({
                "total_step":total_step,
                "loss:":loss.int()
            })
            print("total_step:{} loss:{}".format(total_step,loss_sum/configs.LOSS_RECORD_ITER))
            loss_sum = 0

        # if total_step%configs.CHECKPOINT_ITER == 0:
        #     torch.save(QuackNet.eval(),configs.model_save_dir+"QuackPicker_iter{}.pth".format(total_step))
        #     evaluate(configs.model_save_dir+"QuackPicker_iter{}.pth".format(total_step),"batchsize:{} steps:{}".format(configs.BATCH_SIZE,total_step),eval_dataloader,50,40)

    torch.save(QuackNet.state_dict(),configs.model_save_dir+"QuackPicker_lstm_iter{}.pt".format(total_step))
    QuackNet.train()
evaluate(configs.model_save_dir+"QuackPicker_lstm_iter{}.pt".format(total_step),"batchsize:{} steps:{}".format(configs.BATCH_SIZE,total_step),eval_dataloader,50,45)    
        
        


/root/miniconda3/envs/myconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


total events in csv file: 1265657
total events selected: 574007


/root/miniconda3/envs/myconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


total noise selected: 235426


/root/miniconda3/envs/myconda/lib/python3.8/site-packages/torch/nn/functional.py:3060: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "


total_step:100 loss:698.626513671875
total_step:200 loss:268.1274630737305
total_step:300 loss:147.3941284942627
total_step:400 loss:133.94485916137694
total_step:500 loss:129.251608505249
total_step:600 loss:125.38962203979492
total_step:700 loss:122.23932121276856
total_step:800 loss:118.26866340637207
total_step:900 loss:117.07665718078613
total_step:1000 loss:115.92037391662598
total_step:1100 loss:117.03056129455567
total_step:1200 loss:114.65626609802246
total_step:1300 loss:114.2686882019043
total_step:1400 loss:114.4328335571289
total_step:1500 loss:112.81396850585938
total_step:1600 loss:111.16525482177734
total_step:1700 loss:111.86357940673828
total_step:1800 loss:109.71516159057617
total_step:1900 loss:110.51110496520997
total_step:2000 loss:108.8984913635254
total_step:2100 loss:109.43291130065919
total_step:2200 loss:109.9941584777832
total_step:2300 loss:108.49214881896972
total_step:2400 loss:108.85925941467285
total_step:2500 loss:108.4063631439209
total_step:2600 loss

RuntimeError: Error(s) in loading state_dict for ResUnet_LSTM:
	Unexpected key(s) in state_dict: "res_down_4.BN_1.weight", "res_down_4.BN_1.bias", "res_down_4.BN_1.running_mean", "res_down_4.BN_1.running_var", "res_down_4.BN_1.num_batches_tracked", "res_down_4.BN_2.weight", "res_down_4.BN_2.bias", "res_down_4.BN_2.running_mean", "res_down_4.BN_2.running_var", "res_down_4.BN_2.num_batches_tracked", "res_down_4.conv_1.weight", "res_down_4.conv_1.bias", "res_down_4.conv_2.weight", "res_down_4.conv_2.bias", "res_down_4.bottleneck_conv.weight", "res_down_4.bottleneck_conv.bias", "res_down_5.BN_1.weight", "res_down_5.BN_1.bias", "res_down_5.BN_1.running_mean", "res_down_5.BN_1.running_var", "res_down_5.BN_1.num_batches_tracked", "res_down_5.BN_2.weight", "res_down_5.BN_2.bias", "res_down_5.BN_2.running_mean", "res_down_5.BN_2.running_var", "res_down_5.BN_2.num_batches_tracked", "res_down_5.conv_1.weight", "res_down_5.conv_1.bias", "res_down_5.conv_2.weight", "res_down_5.conv_2.bias", "res_do

## 评估

In [ ]:
from codes.visualize_and_evaluate import evaluate

model_path = ''
evaluate(configs.model_save_dir+"QuackPicker_iter{}.pth".format(total_step),"batchsize:{} steps:{}".format(configs.BATCH_SIZE,total_step),eval_dataloader,50,45)

## 保存损失变化曲线

In [ ]:
header = ["total_step","loss:"]
df_loss = pd.DataFrame(loss_list,columns=header,index=None)
eva_df_old.append(eva_df_new).to_csv(configs.evaluation_csv_path,index=None)